<a href="https://colab.research.google.com/github/TadrousSamir/project/blob/master/9_5_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers torch torchvision torchaudio faiss-cpu tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 99.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 72.1 MB/s eta 0:00:00


In [ ]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch
from transformers import CLIPProcessor, CLIPModel
import faiss
from sklearn.preprocessing import normalize
import re

In [ ]:
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
model.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

CLIPModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 512)
      (position_embedding): Embedding(77, 512)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPSdpaAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (layer_norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=512, out_features=2048, bias=True)
            (fc2): Linear(in_features=2048, out_features=512, bias=True)
          )
          (layer_norm2): LayerNorm((512,), eps=1e

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving final_updated_merged_data.csv to final_updated_merged_data.csv


In [ ]:
df_new = pd.read_csv("final_updated_merged_data.csv")

In [ ]:
DATA_FILE = "final_updated_merged_data.csv"
assert os.path.exists(DATA_FILE), "ملف البيانات غير موجود!"

df = pd.read_csv(DATA_FILE)

In [ ]:
columns_to_use = [
    "gender", "masterCategory", "articleType", "baseColour",
    "season", "year", "usage", "productDisplayName", "Price", "Description"
]

def create_combined_text(row):
    texts = []
    for col in columns_to_use:
        value = str(row.get(col, "")).strip()
        if value and value.lower() != "nan":
            texts.append(value)
    return " ".join(texts)

# تجهيز النصوص لكل المنتجات
combined_texts = df.apply(create_combined_text, axis=1).tolist()

In [ ]:
def get_text_embeddings(texts, batch_size=32):
    embeddings = []
    for i in tqdm(range(0, len(texts), batch_size), desc="تحويل النصوص إلى Embeddings"):
        batch_texts = texts[i:i+batch_size]
        inputs = processor(text=batch_texts, return_tensors="pt", padding=True, truncation=True)
        with torch.no_grad():
            text_features = model.get_text_features(**inputs)
        embeddings.append(text_features.cpu().numpy())
    return np.vstack(embeddings)

text_embeddings = get_text_embeddings(combined_texts)
text_embeddings = normalize(text_embeddings)  # تطبيع لثبات النتائج


تحويل النصوص إلى Embeddings: 100%|██████████| 1389/1389 [40:56<00:00,  1.77s/it]


In [ ]:
def create_faiss_index(embeddings):
    index = faiss.IndexFlatIP(embeddings.shape[1])  # نستخدم inner product لأن التطبيع يجعلها Cosine
    index.add(embeddings)
    return index

index = create_faiss_index(text_embeddings)

In [ ]:
# def search(query, top_k=10):
#     # تجهيز الاستعلام
#     inputs = processor(text=[query], return_tensors="pt", padding=True, truncation=True)
#     with torch.no_grad():
#         query_features = model.get_text_features(**inputs)
#     query_features = normalize(query_features.cpu().numpy())

#     # البحث في FAISS
#     similarities, indices = index.search(query_features, top_k)

#     # طباعة النتائج
#     print(f"\nنتائج البحث عن: {query}\n")
#     for idx, sim in zip(indices[0], similarities[0]):
#         product_id = df.iloc[idx]['id']
#         product_name = df.iloc[idx]['productDisplayName']
#         print(f"ID: {product_id} | Similarity: {sim:.2f} | Name: {product_name}")

In [ ]:
# import time

# queries = [



#     "warm gloves waterproof for skiing trips winter",

# ]

# times = []

# for query in queries:
#     start = time.time()
#     search(query, top_k=10)  # دالتك اللي كتبناها قبل كده
#     end = time.time()
#     times.append(end - start)

# print(f"\n✅ Average time per query: {np.mean(times):.2f} seconds")
# print(f"⏳ Max time for a query: {np.max(times):.2f} seconds")
# print(f"⚡ Min time for a query: {np.min(times):.2f} seconds")

Embedding Queries: 100%|██████████| 2/2 [00:00<00:00,  2.58it/s]



✅ وقت البحث FAISS بس = 0.47 ثانية

🔥 الوقت الكلي للبحث (تحويل+بحث) لـ 1000 Query = 1.77 ثانية


In [ ]:
import time
import torch
import numpy as np
from tqdm import tqdm

def batch_search(queries, top_k=10, batch_size=512, search_batch_size=5000):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    all_embeddings = []

    for i in tqdm(range(0, len(queries), batch_size), desc="تحويل الاستعلامات إلى Embeddings"):
        batch_queries = queries[i:i+batch_size]
        inputs = processor(text=batch_queries, return_tensors="pt", padding=True, truncation=True)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            batch_features = model.get_text_features(**inputs)

        batch_features = normalize(batch_features.cpu().numpy())
        all_embeddings.append(batch_features)

    query_embeddings = np.vstack(all_embeddings)

    all_similarities = []
    all_indices = []

    start_search = time.time()
    for i in tqdm(range(0, len(query_embeddings), search_batch_size), desc="البحث في FAISS"):
        batch_query_embeddings = query_embeddings[i:i+search_batch_size]
        similarities, indices = index.search(batch_query_embeddings, top_k)
        all_similarities.append(similarities)
        all_indices.append(indices)
    end_search = time.time()

    similarities = np.vstack(all_similarities)
    indices = np.vstack(all_indices)

    print(f"\n✅ وقت البحث فقط: {(end_search - start_search):.2f} ثانية")

    # 🔥 طباعة النتائج لكل Query
    for query, sims, idxs in zip(queries, similarities, indices):
        print(f"\nنتائج البحث عن: {query}\n")
        for idx, sim in zip(idxs, sims):
            product_id = df.iloc[idx]['id']
            product_name = df.iloc[idx]['productDisplayName']
            print(f"ID: {product_id} | Similarity: {sim:.2f} | Name: {product_name}")

    return similarities, indices


# --------------------

sample_queries = ["warm gloves waterproof for skiing trips winter"] * 1000

start_total = time.time()
batch_search(sample_queries, top_k=10, batch_size=512)
end_total = time.time()

print(f"\n🔥 the time of 1000 Query = {(end_total - start_total):.2f} ثانية")


البحث في FAISS: 100%|██████████| 1/1 [00:00<00:00,  2.34it/s]


Streaming output truncated to the last 5000 lines.
ID: 53598 | Similarity: 0.64 | Name: Q&Q Men Black Digital Watch
ID: 19441 | Similarity: 0.63 | Name: Quiksilver Men Casual Black Gloves
ID: 13280 | Similarity: 0.63 | Name: Inc 5 Women Casual Multi Flats
ID: 45002 | Similarity: 0.63 | Name: Maxima Men Attivo Digital Watch
ID: 1923 | Similarity: 0.62 | Name: Kalenji Women Running Shoe
ID: 22261 | Similarity: 0.62 | Name: Inc.5 Women Multi Casual Flats

نتائج البحث عن: warm gloves waterproof for skiing trips winter

ID: 53599 | Similarity: 0.64 | Name: Q&Q Men Black Digital Watch
ID: 19442 | Similarity: 0.64 | Name: Quiksilver Men Black Octo Gloves
ID: 11131 | Similarity: 0.64 | Name: Puma Men Beat Black Watches
ID: 13150 | Similarity: 0.64 | Name: Inc 5 Women Casual Multi Heels
ID: 53598 | Similarity: 0.64 | Name: Q&Q Men Black Digital Watch
ID: 19441 | Similarity: 0.63 | Name: Quiksilver Men Casual Black Gloves
ID: 13280 | Similarity: 0.63 | Name: Inc 5 Women Casual Multi Flats
ID: 45

In [23]:
!apt-get install git


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.12).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [24]:
!git config --global user.name "TadrousSamir"
!git config --global user.email "Tadroussamir2001@gmajl.com"

In [26]:
!git clone https://github.com/TadrousSamir/Ai-Model.git


Cloning into 'Ai-Model'...
fatal: could not read Username for 'https://github.com': No such device or address


In [ ]:
# user_query = "looking elegant blak dres for an evening weding party in wint, pric betwen 50 and 150 dolars"
# search(user_query, top_k=20)


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument index in method wrapper_CUDA__index_select)